In [11]:
from battle import *
from entities.trainer import *
from entities.pokemon import *
from moves.moves import *
from globals import *
from entities.team import *
from events.items import *
from events.statuses import *
from events.weathers import *
from battle.battle import *

In [12]:
moves=[Tackle(), Freeze(), Thunder(), Burn(), Earthquake(), ThunderWave(), Rest(), RainDance(), WaterLance(), SwordsDance(), FireBall(), FireSwipe(), Conflagration()]
pikachuStats=Stats(200,100,100,100,100,100)
pikachus=[Pikachu('pika1', 15, pikachuStats, moves)]#, Pikachu('pika2', 15, pikachuStats, moves), Pikachu('pika3', 15, pikachuStats, moves), Pikachu('pika4', 15, pikachuStats, moves)]

ratStats=Stats(200,100,100,100,100,101)
rats=[Rattata('rat1', 20, ratStats, moves)]#, Rattata('rat2', 20, ratStats, moves), Rattata('rat3', 20, ratStats, moves), Rattata('rat4', 20, ratStats, moves)]
rats[0].item=Sword(rats[0])

bellStats=Stats(200, 75, 100, 150, 150, 30)
bell=Bell('Bong', 15, bellStats, moves, None, None)

shroomHogStats=Stats(150, 100, 75, 150, 150, 75)
shroomHog=Shroomhog('SadHog', 15, shroomHogStats, moves)
shroomHog.item=Sword(shroomHog)
t1=Trainer('Trainer 1', [bell, pikachus[0]], None)
t2=Trainer('Trainer 2', [rats[0], shroomHog], None)
teams=[Team('Team Bell', [t1], 1), Team('Team Shrooms', [t2], 1)]
testBattle=Battle(teams)

In [13]:
testBattle.runBattle()
# TODO: sync pokemon fainted status with corresponding slot being empty
# TODO: only show trainers when swapping and sending out pokemon if they have pokemon available

BEING SCHEDULED
EVENTQUEUE SCHEDULE 123
EVENTQUEUE SCHEDULE 123
EVENTQUEUE SCHEDULE 1234
BEING SCHEDULED
EVENTQUEUE SCHEDULE 123
EVENTQUEUE SCHEDULE 123
EVENTQUEUE SCHEDULE 1234


AttributeError: module '__main__' has no attribute '__file__'

In [ ]:
# Have EventScheduler own the event durations and start turns, but pass in those fields into the eventContext object to be used when needed in the event triggers
# Use EventActions to create a buffer between the event itself and the eventQueue

# Bake durations, start turn, and procs into the event itself
# Then we keep all events in a list and give each event a unique ID
# This way when we want to decrease the duration of an event we can just remove 1 from that event easily
# If we want to delete an event from the schedule we can just search for that ID
# This avoids having to handle complex logic to ensure all trigger metadata is matched up

from eventQueue.eventQueue import *
from collections import defaultdict
from collections.abc import Iterable

class EventSystem:
    def __init__(self):
        self.scheduledEvents=defaultdict(list)
        self.eventQueue=EventQueue()

    def schedule(self, event, battleContext, duration=1, startTurn=None, triggers=None):
        if startTurn is None:
            startTurn=battleContext.turn
        if triggers is None:
            triggers=event.triggers
        
        for trigger in triggers:
            self.scheduledEvents[trigger].append([event, duration, startTurn])

    def unschedule(self, shouldRemoveFn, triggers):
        if not isinstance(triggers, Iterable):
            triggers=[triggers]
        for trigger in triggers:
            self.scheduledEvents[trigger]=[e for e in self.scheduledEvents[trigger] if shouldRemoveFn(e)]

    def populateQueue(self, battleContext, trigger):
        for event, duration, startTurn in self.scheduledEvents[trigger]:
            if startTurn==battleContext.turn:
                self.eventQueue.add(event, startTurn, trigger)

    def activateTrigger(self, battleContext, eventContext, trigger):
        self.eventQueue.trigger(battleContext, eventContext, trigger)
        if trigger==Trigger.END_TURN:
            for trig, events in self.scheduledEvents:
                newEvents=[]
                for event, duration, startTurn in events:
                    if startTurn==battleContext.turn:
                        newEvents.append([event, duration-1, startTurn+1])
                self.scheduledEvents[trig]=newEvents
